## SLT Trainer

#### DataLoader

In [1]:
import sys
sys.path.insert(0, './Dataloader')
from SLT_dataloader import EEGROIDataset, SignalDataCollator

# Usage example
roi_folder = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\processed_setfile"
eeg_folder = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\setfile"
group_file = "./Dataloader/subject_groups.json"

# Create dataset
train_dataset = EEGROIDataset(roi_folder, eeg_folder, group_file, "test_train")
print(f"Total dataset size: {len(train_dataset)}")
# Create dataset
test_dataset = EEGROIDataset(roi_folder, eeg_folder, group_file, "test_eval")
print(f"Total dataset size: {len(test_dataset)}")

['314']
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\314_ICA_DLtrain.fdt
Reading 0 ... 77475  =      0.000 ...   302.637 secs...
Total dataset size: 151
['49']
Reading G:\共用雲端硬碟\CNElab_黎承宣&賴璁毅_EEG_ROI\A.Dataset\setfile\49_ICA_DLtrain.fdt
Reading 0 ... 59760  =      0.000 ...   233.438 secs...


g:\共用雲端硬碟\CNElab_陳昱祺\multi-modal\Dataloader\SLT_dataloader.py:56: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_data = mne.io.read_raw_eeglab(eeg_path, preload=True).get_data()


Total dataset size: 114


#### Model

In [ ]:
import sys
sys.path.insert(0, './FirstMultiModel/EEGART')
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from SLT_dataloader import SignalDataCollator

from tf_config import SLTConfig
from tf_model import SLTModel

# 假設你的設備是 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

slt_config = SLTConfig(src_channel_size=30, tgt_channel_size=3*5003, N=2, d_model = 1024)
slt_config.save_pretrained("test_slt_confit")

slt_model = SLTModel(slt_config)
slt_model = slt_model.to(device)


def compute_metrics(eval_preds):
    predictions, targets = eval_preds
    logits = torch.tensor(predictions, dtype=torch.float32) 
    labels = torch.tensor(targets, dtype=torch.float32)

    if torch.isnan(logits).any() or torch.isinf(logits).any():
        print("logits contains nan or inf")
    if torch.isnan(labels).any() or torch.isinf(labels).any():
        print("labels contains nan or inf")
    
    loss_fct = nn.MSELoss()
    
    # Compute the z-scores
    logits_mean = torch.mean(logits, dim=0, keepdim=True)
    logits_std = torch.std(logits, dim=0, keepdim=True)
    logits_norm = (logits - logits_mean) / (logits_std + 1e-10)

    labels_mean = torch.mean(labels, dim=0, keepdim=True)
    labels_std = torch.std(labels, dim=0, keepdim=True)
    labels_norm = (labels - labels_mean) / (labels_std + 1e-10)

    if torch.isnan(logits_norm).any() or torch.isinf(logits_norm).any():
        print("logits_norm contains nan or inf")
    if torch.isnan(labels_norm).any() or torch.isinf(labels_norm).any():
        print("labels_norm contains nan or inf")

    mse = loss_fct(logits_norm.float(), labels_norm.float())
    return {"mse": mse.item()}


training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=48,
    per_device_eval_batch_size=48,
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=True,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

# 初始化模型和 Trainer
trainer = Trainer(
    model=slt_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=SignalDataCollator(),
    compute_metrics=compute_metrics,
)

# 开始训练
trainer.train()

  0%|          | 0/4 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_mmm': nan, 'eval_runtime': 34.1329, 'eval_samples_per_second': 3.34, 'eval_steps_per_second': 0.088, 'epoch': 1.0}
{'train_runtime': 80.0501, 'train_samples_per_second': 1.886, 'train_steps_per_second': 0.05, 'train_loss': 1.897353172302246, 'epoch': 1.0}


TrainOutput(global_step=4, training_loss=1.897353172302246, metrics={'train_runtime': 80.0501, 'train_samples_per_second': 1.886, 'train_steps_per_second': 0.05, 'total_flos': 0.0, 'train_loss': 1.897353172302246, 'epoch': 1.0})